In [2]:
import csv
import numpy as np
import pandas as pd
import json

In [14]:
KDs = pd.read_csv("Data/CFSKDsForVisualization.csv")
Modules = pd.read_csv("Data/CFSModulesForVisualization.csv")
Network = pd.read_csv("Data/CFSNetworkForVisualization_withoutNaN.csv")

### Modules of interest

1) NKM4

2) BcellM7

3) MonoM6

4) NKM13

5) NKM9

In [15]:
Modules

,Unnamed: 0,Module,Features,GeneName,CellType,Unnamed: 5,Unnamed: 6
0,1,BcellM10,ENSG00000204389,HSPA1A,Bcell,NaN,NaN
1,2,BcellM10,ENSG00000114021,NIT2,Bcell,NaN,NaN
2,3,BcellM10,ENSG00000126773,PCNXL4,Bcell,NaN,NaN
3,4,BcellM10,ENSG00000010404,IDS,Bcell,NaN,NaN
4,5,BcellM10,ENSG00000125637,PSD4,Bcell,NaN,NaN
...,...,...,...,...,...,...,...
25678,25679,TempusM1,ENSG00000100591,AHSA1,Tempus,NaN,NaN
25679,25680,TempusM1,ENSG00000230099,TRBV5-4,Tempus,NaN,NaN
25680,25681,TempusM1,ENSG00000089220,PEBP1,Tempus,NaN,NaN
25681,25682,TempusM1,ENSG00000132383,RPA1,Tempus,NaN,NaN


In [16]:
NKM4 = []
BcellM7 = []
MonoM6 = []
NKM13 = []
NKM9 = []

for i in range(len(Modules)):
    if Modules["Module"][i] == "NKM4":
        NKM4.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "BcellM7":
        BcellM7.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "MonoM6":
        MonoM6.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM13":
        NKM13.append(Modules["GeneName"][i])
    if Modules["Module"][i] == "NKM9":
        NKM9.append(Modules["GeneName"][i])  

In [17]:
print(len(NKM4), len(BcellM7), len(MonoM6), len(NKM13), len(NKM9))

92 777 611 202 304


## Nodes creation

In [81]:
parents = np.unique(list(Network["ParentGeneName"]))
children = np.unique(list(Network["ChildGeneName"]))

nodes = list(parents) + list(children)
nodes = np.unique(nodes)

In [82]:
nodes_and_groups = {}

for i in nodes:
    if i in NKM4:
        nodes_and_groups[i] = "NKM4"
    elif i in BcellM7:
        nodes_and_groups[i] = "BcellM7"
    elif i in MonoM6:
        nodes_and_groups[i] = "MonoM6"
    elif i in NKM13:
        nodes_and_groups[i] = "NKM13"
    elif i in NKM9:
        nodes_and_groups[i] = "NKM9"
    else:
        nodes_and_groups[i] = "none"

## Links creation

In [83]:
links = []
for parent, child in zip(Network["ParentGeneName"], Network["ChildGeneName"]):
    links.append([parent, child])

In [84]:
final = {"nodes": [], "links": []}

for node in nodes_and_groups:
    final["nodes"] += [{"id": node, "group": nodes_and_groups[node]}]

for parent, child in links:
    final["links"] += [{"source": parent, "target": child, "value": 1}]

In [85]:
with open('complete_data.json', 'w') as fp:
    json.dump(final, fp)

________

# Modules

### NKM4


In [78]:
## If parent in the group, then we render its child too. 
nkm4_parents = []
nkm4_children = []
nkm4_links = []

for parent, child in links:
    if parent in NKM4:
        nkm4_parents.append(parent)
        nkm4_children.append(child)
        nkm4_links.append([parent, child])

nkm4_parents = np.unique(nkm4_parents)
nkm4_children = np.unique(nkm4_children)

### BcellM7

In [105]:
BcellM7_KD = {}
for kd, gene in zip(KDs["BcellM7_KD"], KDs["geneName_KD"]):
    BcellM7_KD[gene] = kd

len(BcellM7_KD), len(nodes)

(904, 13306)

In [106]:
## THERE IS NOT INFORMATION ABOUT KD FOR ALL NODES. I SET THEM TO 0 IF NON EXISTENT IN THE KD DOCUMENT

for n in nodes:
    if n not in BcellM7_KD:
        BcellM7_KD[n] = 0

len(BcellM7_KD), len(nodes)

(13308, 13306)

In [107]:
links_BcellM7 = []
nodes_BcellM7 = []

for parent, child in links:
    if nodes_and_groups[parent] == "BcellM7" or nodes_and_groups[child] == "BcellM7":
        links_BcellM7.append([parent, child])
        nodes_BcellM7.append([parent, nodes_and_groups[parent], BcellM7_KD[parent]])
        nodes_BcellM7.append([child, nodes_and_groups[child], BcellM7_KD[child]])      

In [111]:
BcellM7_final = {"nodes": [], "links": []}

for node, group, kd in nodes_BcellM7:
    BcellM7_final["nodes"] += [{"id": node, "group": group, "kd": kd}]

for parent, child in links:
    BcellM7_final["links"] += [{"source": parent, "target": child, "value": 1}]

In [112]:
BcellM7_final

{'nodes': [{'id': 'FAM214B', 'group': 'BcellM7', 'kd': 0},
  {'id': 'C17orf103', 'group': 'none', 'kd': 0},
  {'id': 'FAM214B', 'group': 'BcellM7', 'kd': 0},
  {'id': 'WDR45', 'group': 'none', 'kd': 0},
  {'id': 'IL32', 'group': 'none', 'kd': 0},
  {'id': 'CD2', 'group': 'BcellM7', 'kd': 0},
  {'id': 'RPS20', 'group': 'none', 'kd': 0},
  {'id': 'SNN', 'group': 'BcellM7', 'kd': 0},
  {'id': 'TRAF3IP3', 'group': 'MonoM6', 'kd': 0},
  {'id': 'KIAA0907', 'group': 'BcellM7', 'kd': 0},
  {'id': 'TRAF3IP3', 'group': 'MonoM6', 'kd': 0},
  {'id': 'GLUL', 'group': 'BcellM7', 'kd': 0},
  {'id': 'STAB1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'ASGR2', 'group': 'BcellM7', 'kd': 0},
  {'id': 'STAB1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'CARD9', 'group': 'BcellM7', 'kd': 0},
  {'id': 'STAB1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'TBC1D2', 'group': 'BcellM7', 'kd': 0},
  {'id': 'STAB1', 'group': 'BcellM7', 'kd': 0},
  {'id': 'GPR162', 'group': 'none', 'kd': 0},
  {'id': 'CD4', 'group': 'BcellM7',

In [113]:
with open('BcellM7_final.json', 'w') as fp:
    json.dump(BcellM7_final, fp)